<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/xclip_video_classification_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#X-clip video processing: https://arxiv.org/pdf/2207.07285

In [ ]:
!pip -q install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 18.3 MB/s eta 0:00:00


In [ ]:
!pwd

/content


In [ ]:
import os

# Set a custom Hugging Face home directory
os.environ['HF_HOME'] = '/content'

#Extracting Video Features:(8 frames by default)

In [ ]:
import av
import torch
import numpy as np

from transformers import AutoProcessor, AutoModel
from huggingface_hub import hf_hub_download

np.random.seed(0)


def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    '''
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    '''
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices


# video clip consists of 300 frames (10 seconds at 30 FPS)
file_path = hf_hub_download(
    repo_id="nielsr/video-demo", filename="eating_spaghetti.mp4", repo_type="dataset"
)
container = av.open(file_path)

# sample 8 frames
indices = sample_frame_indices(clip_len=8, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
video = read_video_pyav(container, indices)
print('video size:', video.shape)

processor = AutoProcessor.from_pretrained("microsoft/xclip-base-patch32")
model = AutoModel.from_pretrained("microsoft/xclip-base-patch32")

inputs = processor(videos=list(video), return_tensors="pt")

video_features = model.get_video_features(**inputs)
print('extracted video features size:', video_features.shape)

video size: (8, 360, 640, 3)
extracted video features size: torch.Size([1, 512])


#Extracting Frame-wise Feature (8 frames by default)

In [ ]:
import av
import torch
import numpy as np

from transformers import AutoProcessor, XCLIPVisionModel
from huggingface_hub import hf_hub_download

np.random.seed(0)


def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    '''
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    '''
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices


# video clip consists of 300 frames (10 seconds at 30 FPS) https://huggingface.co/datasets/nielsr/video-demo/blob/main/eating_spaghetti.mp4
file_path = hf_hub_download(
    repo_id="nielsr/video-demo", filename="eating_spaghetti.mp4", repo_type="dataset"
)
container = av.open(file_path)

# sample 16 frames
indices = sample_frame_indices(clip_len=8, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
video = read_video_pyav(container, indices)
print('video size:', video.shape)

processor = AutoProcessor.from_pretrained("microsoft/xclip-base-patch32")
model = XCLIPVisionModel.from_pretrained("microsoft/xclip-base-patch32")

pixel_values = processor(videos=list(video), return_tensors="pt").pixel_values

batch_size, num_frames, num_channels, height, width = pixel_values.shape
pixel_values = pixel_values.reshape(-1, num_channels, height, width)

outputs = model(pixel_values)
last_hidden_state = outputs.last_hidden_state

print('extracted frame wise features size:', last_hidden_state.shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


eating_spaghetti.mp4:   0%|          | 0.00/1.01M [00:00<?, ?B/s]

video size: (8, 360, 640, 3)


preprocessor_config.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/965 [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/4.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/786M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/feature_extraction_utils.py:142: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(value)


extracted frame wise features size: torch.Size([8, 50, 768])


#Extracting Frame-wise Feature (5 frames customized)

In [ ]:
from transformers import XCLIPVisionConfig
from safetensors.torch import load_file

configuration = XCLIPVisionConfig()
configuration.num_frames = 5
# configuration.mit_num_attention_heads = 5


indices = sample_frame_indices(clip_len=5, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
video = read_video_pyav(container, indices)
print('video size:', video.shape)

processor = AutoProcessor.from_pretrained("microsoft/xclip-base-patch32")
model = XCLIPVisionModel(configuration)

state_dict = load_file('/content/hub/models--microsoft--xclip-base-patch32/snapshots/a2e27a78a2b5d802e894b8a1ef14f3a8ce490963/model.safetensors')
model.load_state_dict(state_dict, strict=False)

pixel_values = processor(videos=list(video), return_tensors="pt").pixel_values

batch_size, num_frames, num_channels, height, width = pixel_values.shape
pixel_values = pixel_values.reshape(-1, num_channels, height, width)

outputs = model(pixel_values)
last_hidden_state = outputs.last_hidden_state

print('extracted frame wise features size:', last_hidden_state.shape)

video size: (5, 360, 640, 3)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


extracted frame wise features size: torch.Size([5, 50, 768])


In [ ]:
from transformers import AutoProcessor, XCLIPModel, XCLIPProcessor
from transformers import XCLIPConfig

configuration = XCLIPConfig()
configuration.num_frames = 5
configuration.vision_config.num_frames = 5
configuration.vision_config.mit_num_attention_heads = 5

model_xclip= XCLIPModel.from_pretrained("microsoft/xclip-base-patch32")
# model_xclip= XCLIPModel(configuration)

out = model_xclip.visual_projection(last_hidden_state)
print('Final Video Feature:', out.shape)
# out = model_xclip.mit(out.permute(1,0,2))
# print('Final Video Feature:', out.shape)

Final Video Feature: torch.Size([5, 50, 512])


In [ ]:
out[1].shape

torch.Size([50, 512])

#Extracting Video Feature (5 frames customized)

In [ ]:
from transformers import XCLIPConfig
from safetensors.torch import load_file
from transformers import AutoProcessor, XCLIPModel, XCLIPProcessor

configuration = XCLIPConfig()
configuration.num_frames = 5
configuration.vision_config.num_frames = 5
configuration.vision_config.mit_num_attention_heads = 5


processor = AutoProcessor.from_pretrained("microsoft/xclip-base-patch32")
# processor = XCLIPProcessor(configuration)
model = XCLIPModel(configuration)
state_dict = load_file('/content/hub/models--microsoft--xclip-base-patch32/snapshots/a2e27a78a2b5d802e894b8a1ef14f3a8ce490963/model.safetensors')
model.load_state_dict(state_dict, strict=False)

indices = sample_frame_indices(clip_len=5, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
video = read_video_pyav(container, indices)
print('video size:', video.shape)

inputs = processor(videos=list(video), return_tensors="pt")

video_features = model.get_video_features(**inputs)
print('extracted video features size:', video_features.shape)

ValueError: embed_dim must be divisible by num_heads (got `embed_dim`: 512 and `num_heads`: 5).

In [ ]:
model.visual_projection

Linear(in_features=768, out_features=512, bias=False)

In [ ]:
configuration

XCLIPConfig {
  "initializer_factor": 1.0,
  "logit_scale_init_value": 2.6592,
  "model_type": "xclip",
  "num_frames": 5,
  "projection_dim": 512,
  "prompt_alpha": 0.1,
  "prompt_attention_dropout": 0.0,
  "prompt_hidden_act": "quick_gelu",
  "prompt_layers": 2,
  "prompt_num_attention_heads": 8,
  "prompt_projection_dropout": 0.0,
  "text_config": {
    "model_type": "xclip_text_model"
  },
  "transformers_version": "4.44.2",
  "vision_config": {
    "model_type": "xclip_vision_model",
    "num_frames": 5
  }
}

In [ ]:
model.vision_model.config

XCLIPVisionConfig {
  "attention_dropout": 0.0,
  "drop_path_rate": 0.0,
  "hidden_act": "quick_gelu",
  "hidden_size": 768,
  "image_size": 224,
  "initializer_factor": 1.0,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "mit_hidden_size": 512,
  "mit_intermediate_size": 2048,
  "mit_num_attention_heads": 8,
  "mit_num_hidden_layers": 1,
  "model_type": "xclip_vision_model",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_frames": 8,
  "num_hidden_layers": 12,
  "patch_size": 32,
  "transformers_version": "4.44.2"
}

In [ ]:
model.config

XCLIPConfig {
  "initializer_factor": 1.0,
  "logit_scale_init_value": 2.6592,
  "model_type": "xclip",
  "num_frames": 5,
  "projection_dim": 512,
  "prompt_alpha": 0.1,
  "prompt_attention_dropout": 0.0,
  "prompt_hidden_act": "quick_gelu",
  "prompt_layers": 2,
  "prompt_num_attention_heads": 5,
  "prompt_projection_dropout": 0.0,
  "text_config": {
    "model_type": "xclip_text_model"
  },
  "transformers_version": "4.44.2",
  "vision_config": {
    "model_type": "xclip_vision_model"
  }
}